In [1]:
%load_ext autoreload 
%autoreload 2

In [5]:
import torch
import torch.nn as nn
from torch.autograd import Variable

In [18]:
#https://discuss.pytorch.org/t/how-to-reverse-a-torch-tensor/382
def reverse(x):
    #reverse first dimension
    #first - create a tensor with reversed dims 
    idx = [i for i in range(x.size(0)-1, -1, -1)]
    print(idx)
    idx = torch.LongTensor(idx)

    #create tensor with reversed dims
    inverted_tensor = x.index_select(0, idx)

    return inverted_tensor


In [21]:
x = Variable(torch.randn(10,2,2))

In [23]:
print(x)

tensor([[[ 1.2366,  1.1539],
         [-0.8947,  0.0050]],

        [[-2.4095,  0.5682],
         [-0.0267,  0.7975]],

        [[-0.5344,  0.3525],
         [-0.5454,  0.3094]],

        [[ 0.4955,  0.3478],
         [-1.6660, -1.1047]],

        [[-0.7624, -0.3655],
         [ 0.1530, -1.6876]],

        [[-0.4656, -1.0767],
         [ 0.0476,  1.4975]],

        [[ 1.3065,  0.4898],
         [-0.4896,  0.3587]],

        [[-0.2420,  0.2937],
         [ 0.5833,  1.6592]],

        [[ 1.2828,  1.7128],
         [ 0.9528,  1.4779]],

        [[ 1.3652,  0.6777],
         [-0.5947, -0.2400]]])


In [22]:
reverse(x)

[9, 8, 7, 6, 5, 4, 3, 2, 1, 0]


tensor([[[ 1.3652,  0.6777],
         [-0.5947, -0.2400]],

        [[ 1.2828,  1.7128],
         [ 0.9528,  1.4779]],

        [[-0.2420,  0.2937],
         [ 0.5833,  1.6592]],

        [[ 1.3065,  0.4898],
         [-0.4896,  0.3587]],

        [[-0.4656, -1.0767],
         [ 0.0476,  1.4975]],

        [[-0.7624, -0.3655],
         [ 0.1530, -1.6876]],

        [[ 0.4955,  0.3478],
         [-1.6660, -1.1047]],

        [[-0.5344,  0.3525],
         [-0.5454,  0.3094]],

        [[-2.4095,  0.5682],
         [-0.0267,  0.7975]],

        [[ 1.2366,  1.1539],
         [-0.8947,  0.0050]]])

In [51]:
class biGRU(nn.Module):
    def __init__(self,input_size, hidden_size):
        super(biGRU, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size

        #hand designed, so bidirectional is set to False
        self.GRU_f = nn.GRU(input_size, hidden_size, bidirectional=False)
        self.GRU_b = nn.GRU(input_size, hidden_size, bidirectional=False)

    def forward(self, x):
        #make sure that first dimension is timesteps
        batch_size = x.size(1)
        x_f = x
        x_b = reverse(x)

        h_f = Variable(torch.FloatTensor(1, batch_size, self.hidden_size).zero_())
        h_b = torch.zeros_like(h_f)

        #y_f: (T, B, dim)
        #y_b: (T, B, dim)
        y_f, h_f = self.GRU_f(x_f, h_f)
        y_b, h_b = self.GRU_b(x_b, h_b)

        y = torch.cat((y_f, y_b), 2)

        return y
        

In [59]:
gru = biGRU(20,4)

In [60]:
print(gru)

biGRU(
  (GRU_f): GRU(20, 4)
  (GRU_b): GRU(20, 4)
)


In [61]:
x = torch.randn(10,3,20)

In [62]:
y = gru(x)

[9, 8, 7, 6, 5, 4, 3, 2, 1, 0]


In [64]:
y.size()

torch.Size([10, 3, 8])

In [58]:
x.size()

torch.Size([10, 3, 20])